In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 5.1MB/s 


In [ ]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=12OCYH698KOcNsW2wlxQTYSW7ar2t1b5I' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=12OCYH698KOcNsW2wlxQTYSW7ar2t1b5I" -O '/content/video.zip' && rm -rf /tmp/cookies.txt

In [ ]:
#to download video
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-IoWNP7maywHiBMkfErx0iyjRyD9JKYu' -O vv.mp4

In [ ]:
#to download dat file
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-Ca-r5X8W655Bh0EbuohEizl4XSMv3s3' -O shape_predictor_68_face_landmarks2.dat

--2020-05-26 06:16:23--  https://docs.google.com/uc?export=download&id=1-Ca-r5X8W655Bh0EbuohEizl4XSMv3s3
Resolving docs.google.com (docs.google.com)... 74.125.20.113, 74.125.20.100, 74.125.20.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.20.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iibfsc9k4714rtn4mnsbod2phupc3ier/1590473775000/16309236173951571582/*/1-Ca-r5X8W655Bh0EbuohEizl4XSMv3s3?e=download [following]
--2020-05-26 06:16:28--  https://doc-14-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iibfsc9k4714rtn4mnsbod2phupc3ier/1590473775000/16309236173951571582/*/1-Ca-r5X8W655Bh0EbuohEizl4XSMv3s3?e=download
Resolving doc-14-8k-docs.googleusercontent.com (doc-14-8k-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-14-8k-docs.googleusercontent.com (doc-14-8k-docs.g

In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import cv2
import os
from imutils import face_utils
from tqdm import tqdm
import dlib
import random
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [ ]:
#for dividing video to frame
def frames(path):
    obj = cv2.VideoCapture(path)
    count = 0
    success = 1
    while success:
        obj.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
        success, image = obj.read()
        try:
            cv2.imwrite("/content/images/frame%d.jpg" %count, image)
        except:
            pass
        count += 1

In [ ]:
#remove comment when working with videos
#frames('/content/vv.mp4')  

In [ ]:
#draw landmark with red dot on image
def draw_landmark(filename, faces, predictor):
    pixels = cv2.imread(filename)
    pyplot.imshow(pixels)
    ax = pyplot.gca()
    for i in faces:
        rect = i['box']
        left = rect[0]
        top = rect[1]
        right = left + rect[2]
        bottom = top + rect[3]
        rect = dlib.rectangle(left, top, right, bottom)
        shape = predictor(pixels, rect)
        shape = face_utils.shape_to_np(shape)
        for (x, y) in shape:
            dot = Circle((x, y), radius=1, color='red')
            ax.add_patch(dot)
    pyplot.show()

In [ ]:
#function to return landmark array for each image
def landmark_array(filename, p):
    landmark = []
    predictor = dlib.shape_predictor(p)
    pixels = cv2.imread(filename)
    detector = MTCNN()
    faces = detector.detect_faces(pixels)
    #draw_landmark(filename, faces, predictor)
    rect = ((faces[0])['box'])
    left = rect[0]
    top = rect[1]
    right = left + rect[2]
    bottom = top + rect[3]
    rect = dlib.rectangle(left, top, right, bottom)
    shape = predictor(pixels, rect)
    shape = face_utils.shape_to_np(shape)
    for (x, y) in shape:
        landmark.append(x/pixels.shape[0])
        landmark.append(y/pixels.shape[1])
    return landmark

In [ ]:
#create label array 'y'
def label_array(category):
    if(category == 'angry'):
        return 0
    elif(category == 'calm'):
        return 1
    elif(category == 'disgust'):
        return 2
    elif(category == 'fearful'):
        return 3
    elif(category == 'happy'):
        return 4
    elif(category == 'sad'):
        return 5
    elif(category == 'neutral'):
        return 6
    elif(category == 'surprised'):
        return 7

In [ ]:
def dataset(X, Y):
    path = '/content/content/content/data/test/'
    p = '/content/shape_predictor_68_face_landmarks2.dat'
    for category in tqdm(os.listdir(path)):
        path2 = path + category + '/'
        for itr in tqdm(os.listdir(path2)):
            path3 = path2 + itr + '/'
            for k, file in enumerate(os.listdir(path3)):
                if(k==10):
                    break
                X.append(landmark_array((path3+file), p))
                Y.append(label_array(str(category)))
    return X, Y

In [ ]:
# filling X and Y
X = []
Y = []
X, Y = dataset(X, Y)



  0%|          | 0/8 [00:00<?, ?it/s]


  0%|          | 0/14 [00:00<?, ?it/s]


  7%|▋         | 1/14 [00:18<03:55, 18.13s/it]


 14%|█▍        | 2/14 [00:35<03:36, 18.04s/it]


 21%|██▏       | 3/14 [00:53<03:18, 18.00s/it]


 29%|██▊       | 4/14 [01:11<03:00, 18.04s/it]


 36%|███▌      | 5/14 [01:30<02:42, 18.05s/it]


 43%|████▎     | 6/14 [01:48<02:24, 18.10s/it]


 50%|█████     | 7/14 [02:07<02:08, 18.37s/it]


 57%|█████▋    | 8/14 [02:25<01:49, 18.33s/it]


 64%|██████▍   | 9/14 [02:44<01:31, 18.39s/it]


 71%|███████▏  | 10/14 [03:02<01:13, 18.48s/it]


 79%|███████▊  | 11/14 [03:22<00:56, 18.72s/it]


 86%|████████▌ | 12/14 [03:41<00:37, 18.86s/it]


 93%|█████████▎| 13/14 [04:00<00:18, 18.96s/it]


100%|██████████| 14/14 [04:20<00:00, 18.61s/it]


 12%|█▎        | 1/8 [04:20<30:24, 260.58s/it]


  0%|          | 0/33 [00:00<?, ?it/s]


  3%|▎         | 1/33 [00:19<10:18, 19.32s/it]


  6%|▌         | 2/33 [00:38<09:59, 19.32s/it]


  9%|▉         | 3/33 [00:58<09:42, 19

IndexError: ignored

In [ ]:
# shuffling array's
X = np.array(X)
Y = np.array(Y)
X, Y = shuffle(X, Y, random_state=0)

In [ ]:
print(X.shape)
print(Y.shape)

(833, 136)
(833,)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
Y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)

In [ ]:
#print(cm)

In [ ]:
print(accuracy_score(Y_test, Y_pred))

0.6028708133971292


In [ ]:
print(cm)

[[69  0 10  0]
 [ 0  0  5  0]
 [28  0 57  0]
 [32  0  8  0]]


In [ ]:
import pickle

In [ ]:
with open('modelsvm', 'wb') as file:
    pickle.dump(classifier, file)
    file.close()

In [ ]:
#!cp '/content/drive/My Drive/data_test.zip' '/content'

In [ ]:
#!unzip -qq '/content/data_test.zip'

In [ ]:
print(np.unique(Y))

[1 2 5 6]


In [ ]:
#here code for model loading and testing one image

In [ ]:
import pickle
fil = open('/content/modelsvm', 'rb') 
model_ = pickle.loads(fil.read()) 
fil.close()

In [ ]:
X = []
X.append(landmark_array('/content/frame16.jpg', '/content/shape_predictor_68_face_landmarks2.dat'))

In [ ]:
X = np.array(X)

In [ ]:
Y = model_.predict(X)

In [ ]:
print(Y)

[1]
